In [9]:
import pandas as pd
import json
from datetime import datetime
from time import sleep
from time import sleep
import requests
from random import choice
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
from sklearn.externals.joblib import load

etherscan_wallets = set()

tokenview_tokens = ['o8q8Lei2R5RpHMglx05J',
 'sFrkJ9xm3jhC3yw6Q98h',
 'ZNwkHyiVPTWtOuVfULqn',
 'QshQ8WKKT5XnL2zRFQ8w',
 'rXZD5FpaRW1I5jQxiyTI',
 'o28OzKySi8CYhTZZLHul',
 'vdVt7l111UMFtVubb7OM',
 'JOvDoRKsCy0c5wsy1HuM',
 'iFEVMCFqbz2SrP9X8DwW',
 'rTWYOXLiTWs3paSKZnNX',
 'IDVmBJros0w3ulWn7dI4',
 'TaGcjx6lT0zm2ZpVI6q1',
 '38mapNHS9z3FhBiorCXn',
 'VRB3FuClrl0qzs9yNlJr',
 'wOdr9u1R2UWye08rvfgI',
 'EF2HpqTBeO7HhMI2kipN',
 'n3YH3KAawEGzRug4s5Pp',
 'RQuhjZJDOyQkVNCZcQLX',
 'PTowqCQ0vrS0YBCxW72G',
 'st5Ti3eFp5VvArGKmFKt',
 'cy8K3d8eoy4lwGxyHvMf',
 '4HaVdsu2qB6Lsi5RPT3w',
 'QJM1bRmC9R5XTv4Kwv2m',
 'kCSYz62fQsFiMzvZRcWa',
 'C4sDpHl0ZYzCLZXQu6Se',
 'ItWUhfYgZMnZgK4U7hjy',
 'JHJe4Z4WosuMzewzzk3m',
 'rVR3BTymDcx3yQTqQk7Q',
 'tHkxdqn2GwUuamEejvJo',
 'RSy2oQhcsbGBkjn8KUm9']

model = CatBoostClassifier()
model.load_model("CatBoost.model")

dates = pd.read_csv('dates.csv')

def make_predict(addr):
    addr = addr.lower()
    result = []
    if addr in etherscan_wallets:
        #print('first return: 1')
        return 1
    for i in range(1, 51):
        try:
            print(i)
            cur = json.loads(requests.get(f"https://services.tokenview.io/vipapi/address/eth/{addr}/{i}/50?apikey={choice(tokenview_tokens)}").text)
            if cur["code"] == 404 or cur["code"] == '404':
                break
            result += cur['data'][0]['txs']
        except Exception as E:
            i -= 1
            print(E)
            sleep(5)
    #print(result)
    result = [(addr == el['from'].lower(), el['value'], el['time'], el['to'].lower() if addr == el['from'].lower() else el['from'].lower()) for el in result]
    line = result
    if(len(line) < 1):
        #print('second return: 0')
        return ((1, 0), 0)
    for i in range(len(line)):
        el = list(line[i])
        el[1] = float(el[1])
        line[i] = tuple(el)
        
    tnx = line
    tnx.sort(key=lambda x: x[2], reverse=True)
    
    number_of_tnx = len(tnx)
    is_there_2500 = number_of_tnx == 2500
    number_of_sent_tnx = sum(i[0] for i in tnx)
    number_of_received_tnx = number_of_tnx - number_of_sent_tnx
    ratio_sent_received_tnx = number_of_sent_tnx / (number_of_received_tnx + 1)
    
    total_sent_eth = sum(i[1] for i in tnx if i[0])
    total_received_eth = sum(i[1] for i in tnx if not i[0])
    total_eth_interacted_with = total_sent_eth + total_received_eth
    sent_received_ratio = total_sent_eth / (total_received_eth + 0.00001)
    
    unique_values_sent = len(set(i[1] for i in tnx if i[0]))
    unique_values_received = len(set(i[1] for i in tnx if not i[0]))
    unique_values = len(set(i[1] for i in tnx))
    ratio_unique_values_sent_received = unique_values_sent / (unique_values_received + 1)
    
    avg_value_of_sent_tnx = total_sent_eth / (number_of_sent_tnx + 1)
    avg_value_of_received_tnx = total_received_eth / (number_of_received_tnx + 1)
    avg_value_of_all_tnx = total_eth_interacted_with / (number_of_tnx + 1)
    
    time_diff_btw_first_and_last = (tnx[0][2] - tnx[-1][2])
    the_least_time_diff = min(tnx[i][2] - tnx[i + 1][2] for i in range(len(tnx) - 1)) if len(tnx) >= 2 else 0
    the_biggest_time_diff = max(tnx[i][2] - tnx[i + 1][2] for i in range(len(tnx) - 1)) if len(tnx) >= 2 else 0
    avg_sec_btw_tnx = time_diff_btw_first_and_last / number_of_tnx
    
    sent_times = [i[2] for i in tnx if i[0]]
    received_times = [i[2] for i in tnx if not i[0]]
    time_diff_btw_first_and_last_sent = (sent_times[0] - sent_times[-1]) if sent_times else 0
    time_diff_btw_first_and_last_received = (received_times[0] - received_times[-1]) if received_times else 0
    the_least_time_diff_sent = min([sent_times[i] - sent_times[i + 1] for i in range(len(sent_times) - 1)]) if len(sent_times) >= 2 else 0
    the_biggest_time_diff_sent = max([sent_times[i] - sent_times[i + 1] for i in range(len(sent_times) - 1)]) if len(sent_times) >= 2 else 0
    the_least_time_diff_received = min([received_times[i] - received_times[i + 1] for i in range(len(received_times) - 1)]) if len(received_times) >= 2 else 0
    the_biggest_time_diff_received = max([received_times[i] - received_times[i + 1] for i in range(len(received_times) - 1)]) if len(received_times) >= 2 else 0
    avg_sec_btw_tnx_sent = time_diff_btw_first_and_last_sent / (number_of_sent_tnx + 1)
    avg_sec_btw_tnx_received = time_diff_btw_first_and_last_received / (number_of_received_tnx + 1)
    
    unique_addresses_sent = len(set(i[-1] for i in tnx if i[0]))
    unique_addresses_received = len(set(i[-1] for i in tnx if not i[0]))
    unique_addresses_all = len(set(i[-1] for i in tnx))
    
    
    timestamp = int(tnx[0][2])
    dt_object = datetime.fromtimestamp(timestamp)
    
    if not dates[dates.date == str(dt_object.date())]['value'].empty:
        price_in_usd = dates[dates.date == str(dt_object.date())]['value'].iloc[0] 
    else:
        price_in_usd = loads(get(f'https://min-api.cryptocompare.com/data/pricehistorical?fsym=ETH&tsyms=USD&ts={timestamp}').text)['ETH']['USD']
    
    total_sent_eth_usd = total_sent_eth * price_in_usd
    total_received_eth_usd = total_received_eth * price_in_usd
    total_eth_interacted_with_usd = total_eth_interacted_with * price_in_usd
    avg_value_of_sent_tnx_usd = total_sent_eth_usd / (number_of_sent_tnx + 1)
    avg_value_of_received_tnx_usd = total_received_eth_usd / (number_of_received_tnx + 1)
    avg_value_of_all_tnx_usd = total_eth_interacted_with_usd / (number_of_tnx + 1)
    
    #data = pd.Series([number_of_tnx, is_there_2500, number_of_sent_tnx, number_of_received_tnx, ratio_sent_received_tnx, total_sent_eth, total_received_eth, total_eth_interacted_with, sent_received_ratio, unique_values_sent, unique_values_received, unique_values, ratio_unique_values_sent_received, avg_value_of_sent_tnx, avg_value_of_received_tnx, avg_value_of_all_tnx, time_diff_btw_first_and_last, the_least_time_diff, the_biggest_time_diff, avg_sec_btw_tnx, time_diff_btw_first_and_last_sent, time_diff_btw_first_and_last_received, the_least_time_diff_sent, the_biggest_time_diff_sent, the_least_time_diff_received, the_biggest_time_diff_received, avg_sec_btw_tnx_sent, avg_sec_btw_tnx_received, unique_addresses_sent, unique_addresses_received, unique_addresses_all, total_sent_eth_usd, total_received_eth_usd, total_eth_interacted_with_usd, avg_value_of_sent_tnx_usd, avg_value_of_received_tnx_usd, avg_value_of_all_tnx_usd])
    data = [number_of_tnx, number_of_sent_tnx, number_of_received_tnx, ratio_sent_received_tnx, total_sent_eth, total_received_eth, total_eth_interacted_with, sent_received_ratio, unique_values_sent, unique_values_received, unique_values, ratio_unique_values_sent_received, avg_value_of_sent_tnx, avg_value_of_received_tnx, avg_value_of_all_tnx, time_diff_btw_first_and_last, the_least_time_diff, the_biggest_time_diff, avg_sec_btw_tnx, time_diff_btw_first_and_last_sent, time_diff_btw_first_and_last_received, the_least_time_diff_sent, the_biggest_time_diff_sent, the_least_time_diff_received, the_biggest_time_diff_received, avg_sec_btw_tnx_sent, avg_sec_btw_tnx_received, unique_addresses_sent, unique_addresses_received, unique_addresses_all, total_sent_eth_usd, total_received_eth_usd, total_eth_interacted_with_usd, avg_value_of_sent_tnx_usd, avg_value_of_received_tnx_usd, avg_value_of_all_tnx_usd]
    print(len(data))
    t = time = time_diff_btw_first_and_last

    data += [number_of_tnx/t, number_of_sent_tnx/t, number_of_received_tnx/t, total_sent_eth/t, 
             total_received_eth/t, total_eth_interacted_with/t, unique_values_sent/t, unique_values_received/t,
             unique_values/t, unique_addresses_sent/t, unique_addresses_received/t, 
             unique_addresses_all/t, total_sent_eth_usd/t, total_received_eth_usd/t, 
             total_eth_interacted_with_usd/t]
    print(len(data))
    scaler = load("std_scaler_on_divided_by_time.bin")
    scaled = scaler.transform([pd.Series(data)])
    
    to_model = list(scaled[0]) + [is_there_2500]
    
    return model.predict_proba(to_model), model.predict(to_model)
    

In [10]:
to_check = ['0x3f5CE5FBFe3E9af3971dD833D26bA9b5C936f0bE', '0xAb5801a7D398351b8bE11C439e05C5B3259aeC9B', '0xff438fba21b38980484a57ae1419ec84d3a33ea1', '0x492e2cfe720a69433480835cf34dac08fd50e705', '0x21a0e043ac332a082244617b8d69f83631566b41', '0x21B8d56BDA776bbE68655A16895afd96F5534feD', '0x47CE0C6eD5B0Ce3d3A51fdb1C52DC66a7c3c2936']

In [11]:
data = list()
for i in to_check:
    print(i)
    data.append(make_predict(i))

0x3f5CE5FBFe3E9af3971dD833D26bA9b5C936f0bE
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
36
51
0xAb5801a7D398351b8bE11C439e05C5B3259aeC9B
1


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
36
51
0xff438fba21b38980484a57ae1419ec84d3a33ea1
1


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


0x492e2cfe720a69433480835cf34dac08fd50e705
1
2
36
51
0x21a0e043ac332a082244617b8d69f83631566b41
1


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


2
3
36
51
0x47CE0C6eD5B0Ce3d3A51fdb1C52DC66a7c3c2936
1


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
36
51


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [12]:
data

[(array([0.99508268, 0.00491732]), 0),
 (array([0.99271767, 0.00728233]), 0),
 ((1, 0), 0),
 (array([3.38758312e-04, 9.99661242e-01]), 1),
 (array([0.20047856, 0.79952144]), 1),
 (array([0.05162841, 0.94837159]), 1),
 (array([0.99772901, 0.00227099]), 0)]

In [3]:
make_predict('0xff438fba21b38980484a57ae1419ec84d3a33ea1')

1
second return: 0


0